[![Open In Wolfram Cloud](https://raw.githubusercontent.com/gvarnavi/generative-art-iap/master/PR/wolfram-badge.svg)](https://www.wolframcloud.com/obj/gvarnavi/Published/04X_geometric-l-systems.nb)

# Geometric L-Systems

L-systems or "Lindernmayer Systems" originated from botany and refer to recursive systems used to model variety of organisms using:
- An alphabet
- An axiom (or initial state)
- A set of replacement rules

## Algae growth
Let's look at a simple example first, a two-rule system used to model algae growth:

![algae-growth](https://natureofcode.com/book/imgs/chapter08/ch08_24.png)

- Alphabet: 'A' & 'B'
- Axiom: 'A'
- Rules: 'A'&rarr; 'AB', 'B' &rarr; 'A'

We can implement this in the `wolfram language` as a one-liner.  
We'll use the `StringReplace` function, which does the following:

In [ ]:
?StringReplace

**Tip:** Use `?` to quickly query the documentation about more `Information` on a function.

In [ ]:
StringReplace["ABCD", {"A" -> "AB", "B" -> "A"}]
NestList[StringReplace[#, {"A" -> "AB", "B" -> "A"}] &, "A", 4]

In [ ]:
Column[NestList[StringReplace[#, {"A" -> "AB", "B" -> "A"}] &, "A", 4], Center]

## Turtle Graphics
Most operations of interest can be defined by the following "standard" alphabet:

- F: Move forward by a specified distance & draw a line
- G: Move forward by a specified distance w/o drawing a line
- +: Turn right by a specified angle
- -: Turn left by a specified angle  

This is commonly referred to as 'Turtle graphics'.  
Imagine a turtle sitting on your computer screen following a limited set of commands.

### Geometric Fractals
For simple systems (without branches), our turtle just keeps going in a straight line.
It is fairly straightforward in those cases to use `AnglePath`, a generalization of a random walk.

We'll start by restricting our random walkers to walk on a square grid.

In [ ]:
Tuples[Range[-1, 1], 2]

In [ ]:
(*Only selecting steps with unit legth*)
steps = Select[Tuples[Range[-1, 1], 2], Norm[#] == 1 &]

(*Randomly selecting one at each step*)
latticeStep[] := RandomChoice[steps]

In [ ]:
(*let's select 9 such random steps, starting from the origin*)
Prepend[Table[latticeStep[], 9], {0, 0}]

Finally, we `Accumulate` these steps and plot a line through them

In [ ]:
Prepend[Table[latticeStep[], 1000], {0, 0}]//Accumulate//Line//Graphics

**Coding Comment:** We used Postfix notation (`//`) above.  
The Wolfram Language provides the following equivalent notations for applying functions:  
~~~Mathematica
f[a]
f@a   (*Prefix*)
a//f  (*Postfix*)
a~f~b (*Infix, equivalent to f[a,b]*)
~~~

That means the code above is equivalent to:
~~~Mathematica
Graphics[Line[Accumulate[Prepend[Table[latticeStep[], 1000], {0, 0}]]]]
~~~
You'll find Postfix to be quite useful, kind-of like an afterthought to quickly change your output

We can make a better-looking visualization by using disks and opacity for our visited locations.

In [ ]:
With[{locations= Accumulate[Prepend[Table[latticeStep[], 1000], {0, 0}]]},
Graphics[{Opacity[0.25], 
    Disk[#, 1/2] & /@locations}]]

Let's relax one of those constraints, by allowing our walkers to take unit steps at any angle

In [ ]:
randomStep[] := Module[{\[Theta] = RandomReal[{0, 2 Pi}]},{Cos[\[Theta]], Sin[\[Theta]]}]
Prepend[Table[randomStep[], 1000], {0, 0}] // Accumulate // Line // Graphics

We'll make this periodic inside a square box, and visualize using our disks/opacity scheme above

In [ ]:
With[{steps =Mod[Accumulate[Prepend[Table[randomStep[], 10000], {0, 0}]], 50, -25]}, 
 Rasterize[
  Multicolumn[Table[Graphics[{Opacity[0.25], Disk[#, 1/2] & /@ Take[steps, time]}, 
     PlotRange -> 25 {{-1, 1}, {-1, 1}}], 
     {time,Subdivide[1, 10001, 20]}], 
     7, Appearance -> "Horizontal", 
   Frame -> All]]]

**Note:**
The Manipulate/Animate integration for the Wolfram-Language jupyter kernel is still lacking, so we create a grid of frames instead. We'll use the wolfram cloud notebooks at times, when we need the functionality.

At this point, we can use the built-in `AnglePath` to visualize simple L-systems

In [ ]:
?AnglePath

In [ ]:
RandomReal[{0, 2 \[Pi]}, 1000] // AnglePath // Line // Graphics

We can use the second form of the function to also vary the length of the step

In [ ]:
AnglePath[
   Transpose[{RandomReal[{0, 1}, 1000], 
     RandomReal[{0, 2 \[Pi]}, 1000]}]] // Line // Graphics

We can take advantage of the variable length argument to control drawing/skipping letters and the variable angle argument to take turns

In [ ]:
SubstitutionSystem[{"X" -> "X+YF+", "Y" -> "-FX-Y"}, "FX", {4}][[1]]

In [ ]:
StringCases[%,{"F" -> {1, 0}, "X" -> {0, 0}, "Y" -> {0, 0}, "+" -> {0, Pi/2}, "-" -> {0, -Pi/2}}]

In [ ]:
%//AnglePath//Line//Graphics

**Coding Comment:** We're using `%` to refer to the previous output for brevity

Let's make some visualization functions and see some systems drawn out!

In [ ]:
(*Legend function to graphically illustrate replacement rules*)

legend[rules_, axiom_, angle_, name_] := 
 Block[{rp, flatRules, counter},
  flatRules = 
   Flatten[rules /. (a_ -> b_) :> Characters[StringJoin[a, b]]];
  rp = RulePlot[SubstitutionSystem[rules], Frame -> False, 
    Appearance -> "Arrow", 
    ImageSize -> Clip[40 Length[flatRules], {400, 800}]];
  counter = 1;
  Show[rp /. {col_, 
      Rectangle[min_, max_]} :> {{col, 
       Rectangle[min, max]}, {Text[
        Style[flatRules[[counter++]], 
         Clip[Length[flatRules], {10, 14}], 
         If[col[[1]] == 1., Black, White]], Mean[{min, max}]]}}]]
         
(*Visualization function to nest substitution rules and draw last iteration*)

visualize[name_, {rules_, axiom_, angle_, {drawLetters_, dummyLetters_},iter_}] := 
 Block[{str, label, ruleLegend, fractal},
  str = Last[SubstitutionSystem[rules, axiom, {iter}]];
  fractal = 
   Graphics[
    Line@AnglePath[
      StringCases[str, 
       Join[Thread[{drawLetters -> {1, 0}}], 
        Thread[{dummyLetters -> {0, 0}}], {"+" -> {0, angle}, 
         "-" -> {0, -angle}}]]], ImageSize -> 600,
         
    PlotLabel -> 
     Style[StringTemplate[
        "`1`\nAxiom: `2`\tAngle: `3`\tDraw Letters:`4`"]
        [name, axiom,UnitConvert[Quantity[angle, "Radians"], "Degrees"],
        
       StringRiffle[drawLetters, ","]], 18, Black]];
  
  ruleLegend = legend[rules, axiom, angle, name];
  Legended[fractal, Placed[ruleLegend, Bottom]]]

### Examples
- Dragon Curve
  - Alphabet: F X Y + -
  - Axiom: FX
  - Drawing Letters: F
  - Rules: (X -> X +Y F +) &  (Y ->- F X- Y)
  - Angle : 90 Degrees

In [ ]:
dragonCurve = 
 visualize["Dragon Curve", 
 {{"X" -> "X+YF+", "Y" -> "-FX-Y"}, 
 "FX", \[Pi]/2, 
 {{"F"}, {"X", "Y"}}, 
 12}]

- Sierpinski Triangle
  - Alphabet: F G + -
  - Axiom: F
  - Drawing Letters: F, G
  - Rules: (F -> G + F + G) &  (G -> F - G - F)
  - Angle : 60 Degrees

In [ ]:
sierpinskiTriangle = 
 visualize["Sierpinski Triangle", 
 {{"F" -> "G+F+G", "G" -> "F-G-F"}, 
   "F", \[Pi]/3, 
   {{"F", "G"}, {}}, 8}]

- Diffusion Limited Aggregation
  - Alphabet: F +
  - Axiom: F + F + F + F
  - Drawing Letters: F
  - Rules: F -> F F + F + + F + F
  - Angle : 90 Degrees

In [ ]:
dla = visualize["Diffusion Limited Aggregation", 
{{"F" -> "FF+F++F+F"}, 
   "F+F+F+F", \[Pi]/2, 
   {{"F"}, {}}, 6}]

### Push-Pop Systems
For systems involving push and pop, we need to implement a stack and such write our own visualization function.  
In particular, we'll introduce two new symbols to our rule-set:

- [: Save current location & **optionally** turn right
- ]: Restore previous location & **optionally** turn left

We start by specifying an "algorithmic" way  of specifying rotate right and left.
Essentially we just need a cyclic +/- modifier.

In [ ]:
I^2 (*turn right*)
I^4 (*turn left*)
I^6 (*push and turn right*)
I^8 (*pop and turn left*)

Next, let's write our own nesting function:

In [ ]:
nest[state_, rules_, iter_] := Nest[Flatten[# /. rules] &, state, iter]
nest[{"F", 2, 2, "F", 2, 2,"F"}, {"F" -> {"F", 4, "F", 2, 2, "F", 4, "F"}}, 2]

Finally, we write our visualization function:

In [ ]:
(*Implementating a push-pop style stack*)
visualizeLSystemPushPop[state_, rotAngle_, drawLetters_, rotateOnPushPopQ_:True] := 
 Module[{currentAngle = 0, currentLocation = {0, 0}, 
   currentState = {}, savedState = {}, savedAngle = 0, 
   savedLocation = {0, 0}},
  (Switch[#,
      6,
      savedState = {savedAngle, savedLocation, savedState},
      8,
      {savedAngle, savedLocation, savedState} = savedState,
      
      _?(Or @@ Thread[# == drawLetters] &),
      
      currentState = {currentState, 
        Line@{savedLocation, 
          savedLocation += {Cos@savedAngle, Sin@savedAngle}}}];
     
     If[
     If[rotateOnPushPopQ,
     NumericQ[#],
     #==2||#==4], savedAngle += I^# rotAngle]) & /@ state;
  
  Graphics[Flatten@currentState, ImageSize -> 500]
  ]

(*utility function to convert b/w the numeric and string rule representations*)
convertRules[rules_] := rules /. (a_ -> b_) :> (a -> StringJoin @@ (b /. {2 -> "+", 4 -> "-", 6 -> "[", 8 -> "]"}))

(*Visualization function to nest push-pop substitution rules and draw last iteration*) 
visualizePushPop[name_, {rules_, axiom_, angle_, {drawLetters_, dummyLetters_}, iter_},rotateOnPushPopQ_:True] := 
 Block[{str, label, ruleLegend, fractal},
  fractal = 
   Show[visualizeLSystemPushPop[
     nest[{axiom}, rules, iter], angle,
      drawLetters, rotateOnPushPopQ], ImageSize -> 600, 
    PlotLabel -> 
     Style[StringTemplate[
        "`1`\nAxiom: `2`\tAngle: `3`\tDraw Letters:`4`"][name, axiom, 
       UnitConvert[Quantity[angle, "Radians"], "Degrees"], 
       StringRiffle[drawLetters, ","]], 18, Black]];
       
  ruleLegend = legend[convertRules[rules],axiom, angle, name];
  Legended[fractal, Placed[ruleLegend, Bottom]]]

### Examples
- Fractal Plant
  - Alphabet: X F + - [ ]
  - Axiom: X
  - Drawing Letters: F, X
  - Rules:(X -> F + [ [ X ] - X ] - F [ - F X ] + X) & (F -> F F
  - Angle : 15 Degrees
  - Rotate on Push/Pop : True

In [ ]:
fractalPlant = 
 visualizePushPop["Fractal Plant", 
 {{"X" -> {"F", 4, 6, 6, "X", 8, 2, "X", 8, 2, "F", 6, 2, "F", "X", 8, 4, "X"}, 
 "F" -> {"F", "F"}}, "X", \[Pi]/12, {{"F", "X"}, {}}, 4},True]

- Bush
  - Alphabet: F + - [ ]
  - Axiom: F
  - Drawing Letters: F
  - Rules:F -> FF+[+F-F-F]-[-F+F+F]
  - Angle : 22.5 Degrees
  - Rotate on Push/Pop : False

In [ ]:
bush = 
 visualizePushPop["Bush", 
 {{"F" -> {"F", "F", 2, 6, 2, "F", 4, "F", 4, "F", 8, 4, 6, 4, "F", 2, 
   "F", 2, "F", 8}}, "F", 22.5 Degree, {{"F"}, {}}, 4},False]

## Your turn!
Play around with the `visualize` or `visualizePushPop` functions to come up with your own fractals.  
[Paul Bourke's website](http://paulbourke.net/fractals/lsys/) is an excellent starting point for L-system rules!  
(Note: the website defines push/pop w/o rotation so make sure to pass the Boolean flag `False` as the last argument, if you use rules from there.)

## (Time-permitting) Genetic L-Systems
The fractals we investigated above required very specific rules - can we instead randomly produce them?  
We'll use the [wolfram cloud notebook](https://www.wolframcloud.com/obj/gvarnavi/Published/03X_geometric-l-systems.nb) for this, to allow some interactivity!